## train with custom dataset

### Create a training YAML file

In [ ]:
from functools import total_ordering
import os
import shutil
import random

image_dir = '/content/drive/new_image'
annotation_dir = '/content/drive/label'

base_output_dir = "/content/yolo_pr"
train_image_dir = os.path.join(base_output_dir, "images/train")
val_image_dir = os.path.join(base_output_dir, "images/val")
train_label_dir = os.path.join(base_output_dir, "labels/train")
val_label_dir = os.path.join(base_output_dir, "labels/val")

def create_dirs():
    for d in [train_image_dir, val_image_dir,train_label_dir, val_label_dir]:
        os.makedirs(d, exist_ok=True)

image_files = sorted([f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png', '.jpeg'))])
random.seed(33)
random.shuffle(image_files)

# train : validation = 8:2
total_image = len(image_files)
train_size = int(total_image * 0.8)

train_files = image_files[:train_size]
val_files = image_files[train_size:]

def move_files(file_list, src_img_dir, src_ann_dir, dest_img_dir, dest_ann_dir):
    for file_name in file_list:
        img_src_path = os.path.join(src_img_dir, file_name)
        img_dest_path = os.path.join(dest_img_dir, file_name)

        ann_file_name = os.path.splitext(file_name)[0] + ".txt"
        ann_src_path = os.path.join(src_ann_dir, ann_file_name)
        ann_dest_path = os.path.join(dest_ann_dir, ann_file_name)

        if os.path.exists(img_src_path):
            shutil.copy(img_src_path, img_dest_path)

        if os.path.exists(ann_src_path):
            shutil.copy(ann_src_path, ann_dest_path)

def create_yaml():
    yaml_content = f"""
train: {train_image_dir}
val: {val_image_dir}
nc: 6
names: ['sign','sign_word','direction','hurdle','working_truck','tripod']
"""
    yaml_path = os.path.join(base_output_dir, "dataset.yaml")
    with open(yaml_path, "w") as f:
        f.write(yaml_content)

def main():
    create_dirs()
    move_files(train_files, image_dir, annotation_dir, train_image_dir, train_label_dir)
    move_files(val_files, image_dir, annotation_dir, val_image_dir, val_label_dir)
    create_yaml()
    print("Dataset preparation completed! YAML file created.")

main()

### train

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

model.load('/content/bdd100k_yolo/yolo_results/train/weights/best.pt')
results = model.train(data='/content/yolo_pr/dataset.yaml', epochs=150,patience=10)
results = model.val()
